# What *drugs/compounds* target gene products of gene "coagulation factor II, thrombin"?

In [1]:
!pip install SPARQLWrapper pandas --quiet
from SPARQLWrapper import SPARQLWrapper, JSON, XML, CSV, TSV
import pandas as pd, io
from IPython.core.display import display, HTML

The NCATS Translator [Red team knowledge graph](http://graphdb.dumontierlab.com/repositories/ncats-red-kg) is used to answer the question.

The Red team knowledge graph integrates the following datasets:
* HGNC: genes
* DrugBank: drugs, drug-protein and drug-drug interactions
* PrePPI: protein-protein interactions
* BioGRID: protein-protein interactions with evidences
* WikiPathways: pathways, gene-gene interactions
* DATE: link drugs to pathways
* GOTE: link proteins to pathways

The SPARQL queries used to transform the datasets to the [BioLink model](https://biolink.github.io/biolink-model/docs/) can be found [here](https://github.com/vemonet/data2services-insert/tree/master/insert-biolink).

In [3]:
sparql = SPARQLWrapper("http://graphdb.dumontierlab.com/repositories/ncats-red-kg")
sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bl: <http://w3id.org/biolink/vocab/>
SELECT distinct ?gene ?produceProtein ?affectedByDrug ?byRelation ?inInteraction ?inGraph
{
  ?geneUri a bl:Gene ;
    bl:id ?geneId ; 
    bl:name ?gene ;
    bl:has_gene_product ?geneProductUri .
  OPTIONAL{ ?geneProductUri bl:name ?produceProtein . }
  ?geneProductUri a bl:GeneProduct .
  GRAPH ?inGraph {
      ?inInteraction ?associationSubject ?geneProductUri ;
       ?associationObject ?drugUri ;
       bl:relation ?byRelation .
   }
  ?drugUri a bl:ChemicalSubstance .
  OPTIONAL {?drugUri bl:name ?affectedByDrug .}
} limit 100
""")

sparql.setReturnFormat(CSV)
results = sparql.query().convert()

df = pd.read_csv(io.BytesIO(results))
display(HTML(df.to_html()))
#print(df.to_html())

,gene,produceProtein,affectedByDrug,byRelation,inInteraction,inGraph
0,gamma-aminobutyric acid type A receptor beta2 ...,NaN,compendium,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
1,gamma-aminobutyric acid type A receptor beta2 ...,NaN,ethchlorvynol,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
2,gamma-aminobutyric acid type A receptor beta2 ...,NaN,estazolam,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
3,gamma-aminobutyric acid type A receptor beta2 ...,NaN,diazepam,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
4,gamma-aminobutyric acid type A receptor beta2 ...,NaN,alprazolam,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
5,gamma-aminobutyric acid type A receptor beta2 ...,NaN,adinazolam,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
6,gamma-aminobutyric acid type A receptor beta2 ...,NaN,midazolam,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
7,gamma-aminobutyric acid type A receptor beta2 ...,NaN,compendium,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
8,gamma-aminobutyric acid type A receptor beta2 ...,NaN,ethchlorvynol,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
9,gamma-aminobutyric acid type A receptor beta2 ...,NaN,diazepam,http://w3id.org/biolink/vocab/interacts_with,https://w3id.org/data2services/data/protein/in...,https://w3id.org/data2services/graph/biolink/date
